<a href="https://colab.research.google.com/github/atta007/Plant_Disease_Classifier/blob/master/Plant_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D


#Importing Libraries for Deep learning Model 

Using TensorFlow backend.


In [0]:
# Download the dataset and unzip de folder
!gdown https://zenodo.org/record/1204914/files/plantvillage_deeplearning_paper_dataset.7z?download=1
  
#Downloading Dataset   

Downloading...
From: https://zenodo.org/record/1204914/files/plantvillage_deeplearning_paper_dataset.7z?download=1
To: /content/plantvillage_deeplearning_paper_dataset.7z?download=1
100% 824M/824M [00:14<00:00, 56.8MB/s]


In [0]:
!tar -xvf PlantVillage.tar.gz


#extracting Downloaded Dataset of Plants images 

In [0]:
data_dir = '/content/PlantVillage'   #accessing the directory of Dataset
train_dir = data_dir + '/train'      # spliting the 
valid_dir = data_dir + '/val'


# dimensions of our images.
img_width, img_height = 150, 150
input_shape= (3, img_width, img_height)

epochs = 200
batch_size = 100

nb_train_samples = 4000
nb_validation_samples =  1000

#if K.image_data_format() == 'channels_first':
    #input_shape = (3, img_width, img_height)
#else:
  #input_shape = (img_width, img_height, 3)
  
print(input_shape) 

(3, 150, 150)


In [0]:
from keras import optimizers

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2,2))



model.add(Conv2D(128, (2, 2), activation='relu'))
model.add(MaxPooling2D(2,2))


model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add (Conv2D(256, (1,1), activation ='relu'))
model.add(MaxPooling2D(1,1))





model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(39, activation='softmax'))


#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

from keras.metrics import categorical_accuracy
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
             metrics=['accuracy'])
    


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 146, 146, 32)      2432      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 34, 34, 128)       32896     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 8, 8, 128)         0         
__________

In [0]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.4,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)





In [0]:
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

#print(train_generator.class_indices)

validation_generator = test_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
shuffle=True)

#display Data from generators
#next(train_generator)


#per = np.random.permutation(train_generator.n)
#train_generator.index_array = per 
#classes = train_generator.classes[per]

#print(classes[1])


label_map = (train_generator.class_indices)
print(label_map)

Found 44016 images belonging to 39 classes.
Found 11004 images belonging to 39 classes.
{'Apple___Apple_scab': 0, 'Apple___Black_rot': 1, 'Apple___Cedar_apple_rust': 2, 'Apple___healthy': 3, 'Blueberry___healthy': 4, 'Cherry_(including_sour)___Powdery_mildew': 5, 'Cherry_(including_sour)___healthy': 6, 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 7, 'Corn_(maize)___Common_rust_': 8, 'Corn_(maize)___Northern_Leaf_Blight': 9, 'Corn_(maize)___healthy': 10, 'Grape___Black_rot': 11, 'Grape___Esca_(Black_Measles)': 12, 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 13, 'Grape___healthy': 14, 'Orange___Haunglongbing_(Citrus_greening)': 15, 'Peach___Bacterial_spot': 16, 'Peach___healthy': 17, 'Pepper,_bell___Bacterial_spot': 18, 'Pepper,_bell___healthy': 19, 'Potato___Early_blight': 20, 'Potato___Late_blight': 21, 'Potato___healthy': 22, 'Raspberry___healthy': 23, 'Soybean___healthy': 24, 'Squash___Powdery_mildew': 25, 'Strawberry___Leaf_scorch': 26, 'Strawberry___healthy': 27, 'Tomato

In [0]:
from keras.callbacks import EarlyStopping
from tensorboardcolab import *
tbc = TensorBoardColab()
es = EarlyStopping(monitor = "val_loss", patience = 10)

Wait for 8 seconds...
TensorBoard link:
https://50e63fca.ngrok.io


In [0]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
validation_steps=nb_validation_samples // batch_size,
    callbacks = [es, TensorBoardColabCallback(tbc)]
            )

model.save_weights("Plant_Classifier.h5")
model.save("Plant_Classifier.h5", True)

#score = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])
#print('Scode', score)

Epoch 1/200
40/40 [==============================] - 36s 897ms/step - loss: 2.8638 - acc: 0.2468 - val_loss: 2.3623 - val_acc: 0.3740
Epoch 2/200
40/40 [==============================] - 35s 864ms/step - loss: 2.3143 - acc: 0.3793 - val_loss: 1.8735 - val_acc: 0.4970
Epoch 3/200
40/40 [==============================] - 35s 868ms/step - loss: 1.8280 - acc: 0.4847 - val_loss: 1.6014 - val_acc: 0.5450
Epoch 4/200
40/40 [==============================] - 35s 864ms/step - loss: 1.5919 - acc: 0.5437 - val_loss: 1.3062 - val_acc: 0.6140
Epoch 5/200
40/40 [==============================] - 35s 874ms/step - loss: 1.4187 - acc: 0.5870 - val_loss: 1.3456 - val_acc: 0.6010
Epoch 6/200
40/40 [==============================] - 35s 880ms/step - loss: 1.3735 - acc: 0.6033 - val_loss: 1.1144 - val_acc: 0.6600
Epoch 7/200
40/40 [==============================] - 35s 870ms/step - loss: 1.1873 - acc: 0.6432 - val_loss: 0.9735 - val_acc: 0.6840
Epoch 8/200
40/40 [==============================] - 35s 873ms

In [0]:
score = model.evaluate_generator(validation_generator, nb_validation_samples// batch_size)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Score', score)

Test loss: 0.26152691394090655
Test accuracy: 0.9130000054836274
Score [0.26152691394090655, 0.9130000054836274]


In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
!ls

In [0]:
def predict_image():
# Testing Predicition from Validation Folder
  from keras.models import load_model
  from keras.preprocessing import image
  import numpy as np


  test_image = image.load_img('PlantVillage/val/Tomato___Tomato_Yellow_Leaf_Curl_Virus/33533dfd-f34a-46eb-9896-c55b5cb33258___UF.GRC_YLCV_Lab 09469.JPG', target_size=(img_width, img_height, 3))
  print(test_image)
  test_image = image.img_to_array(test_image)
  test_image = np.expand_dims(test_image, axis=0)

  print(test_image.shape)
  import numpy as np
  from keras.preprocessing import image

  pred_img_class = model.predict(test_image)

  print(type(pred_img_class))
  print(pred_img_class)



  y_classes = pred_img_class.argmax(axis=-1)
  for a, b in label_map.items():
    if b == y_classes[0]:
      print(a)

      print(str(pred_img_class[0,y_classes]*100)+'Percent')

In [0]:
predict_image()


    

<PIL.Image.Image image mode=RGB size=150x150 at 0x7FDF2617F780>
(1, 150, 150, 3)
<class 'numpy.ndarray'>
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]
Tomato___Tomato_Yellow_Leaf_Curl_Virus
[100.]Percent
